## Github Actions and ACT


Table of Contents
* **Triggers and Matrices**
* Variables and Secrets
* Authentication and Deployment
* Local Development with ACT

### Terminology

job - a shell script or action
action - 
workflow - one or more jobs
runner - [ubuntu-lastest]
events - [push, PR, manual trigger]

### Triggers

```yaml
on:
  push:
    branches:
    - main
    - 'releases/**'
  pull_request:
    branches:
    - main
    - 'releases/**'
```

### Variables

```yaml
env:
  TAG_NAME: "tag_${{ github.sha }}_${{ github.run_attempt }}"
```

```yaml
run: |
  git push https://${{ secrets.GITHUB_TOKEN }}@https://github.com/${{ github.repository }} ${{ env.TAG_NAME }}

```

In [ ]:
### Combining them

```yaml
name: Example 1 Workflow
on:
  push:
    branches:
      - main
  workflow_dispatch:
env:
  TAG_NAME: "tag_${{ github.sha }}_${{ github.run_attempt }}"
jobs:
  example1:
    name: Example 1 Job
    runs-on: ubuntu-latest
    steps:
    - name: Example 1 Action
      run: |
        git push https://${{ secrets.GITHUB_TOKEN }}@https://github.com/${{ github.repository }} ${{ env.TAG_NAME }} -m "Automated!"
```

In [2]:
!gh run list --workflow example1.yaml --limit 1

STATUS  NAME          WORKFLOW       BRANCH  EVENT  ID          ELAPSED  AGE
X       added naming  Example 1 ...  main    push   2848359284  16s      5m
